In [ ]:
import pandas as pd
import numpy as np
from itertools import chain
import random
from  sklearn.model_selection import train_test_split

In [ ]:
area_population = pd.read_csv('/content/indian.csv')
area_population.columns

Index(['Rank', 'City', 'Population(2011)', 'Population(2001)',
       'State or union territory'],
      dtype='object')

In [ ]:
zomato = pd.read_csv('/content/zomato.csv', encoding='latin1')


In [ ]:
zomato_ind = zomato[zomato['Country Code'] == 1]
zomato_ind.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [ ]:
restaurant_data = zomato_ind[['Restaurant Name' , 'City' , 'Cuisines' , 'Longitude' ,'Latitude' ,'Aggregate rating' , 'Price range' , 'Has Online delivery']]

In [ ]:
graphics_data = area_population[['City' ,'Population(2011)']]
graphics_data['Avg_Income'] = np.random.randint(1000, 10001, size=len(area_population))
graphics_data['Youth_Percentage'] = np.round(np.random.rand(len(area_population)), 2)


/tmp/ipython-input-3140965880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graphics_data['Avg_Income'] = np.random.randint(1000, 10001, size=len(area_population))
/tmp/ipython-input-3140965880.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  graphics_data['Youth_Percentage'] = np.round(np.random.rand(len(area_population)), 2)


In [ ]:
restaurant_data_cuisines = restaurant_data['Cuisines'].apply(lambda x :[c.strip() for c in x.split(',')])

cuisine_pool = list(set(chain.from_iterable(restaurant_data_cuisines)))


In [ ]:
cities = pd.read_csv('/content/Indian Cities Database.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Indian Cities Database.csv'

In [ ]:
competitor_data = pd.DataFrame({
    'City': cities['City'],
    'Total_Restaurants': np.random.randint(100 , 1500 , size= len(cities)),
    'Avg_Rating': np.random.randint(1 , 6 , size=len(cities)),
    'Popular_Cuisines': [random.sample(cuisine_pool, 2 )for _ in range(len(cities))]
})


In [ ]:
location_data = pd.DataFrame({
    'City': cities['City'],
    'Foot_Traffic_Index': np.random.randint(1 , 101 , size=len(cities)),
    'Avg_Rent_per_sqft': np.random.randint(10 , 101 , size=len(cities))
})

merge tha Datasets for training

In [ ]:
finaldata = restaurant_data.merge(graphics_data , left_on='City' , right_on='City' )
finaldata = finaldata.merge(location_data  , left_on='City' , right_on='City' )
finaldata = finaldata.merge(competitor_data  , left_on='City' , right_on='City' )
finaldata['Review_Count'] = np.random.randint(20 , 500 , size= len(finaldata))


In [ ]:
finaldata['Population(2011)'] = finaldata['Population(2011)'].str.replace(r'\D', '', regex=True).astype(int)
# finaldata['Population(2011)']

In [ ]:
finaldata['Population(2011)'].dtypes

In [ ]:
finaldata.columns

In [ ]:
# Competition Index
finaldata['Competition_Index'] = finaldata['Total_Restaurants'] / finaldata['Population(2011)']

# Affordability
finaldata['Affordability'] = finaldata['Avg_Income'] / finaldata['Avg_Rent_per_sqft']

# Popularity Index
finaldata['Popularity_Index'] = finaldata['Aggregate rating'] * finaldata['Review_Count'] / finaldata['Total_Restaurants']

# Target Score (for supervised learning)
finaldata['Success_Score'] = (
    finaldata['Aggregate rating'] * 0.4 +
    (finaldata['Review_Count'] / finaldata['Review_Count'].max()) * 0.2 +
    (finaldata['Foot_Traffic_Index'] / 100) * 0.2 +
    (finaldata['Affordability'] / finaldata['Affordability'].max()) * 0.2
)


In [ ]:
finaldata.columns

In [ ]:
finaldata.head()

In [ ]:
# finaldata = pd.get_dummies(finaldata, columns=['Cuisine'], drop_first=True)
finaldata.dtypes


In [ ]:
finaldata = pd.get_dummies(finaldata, columns=['Cuisines'], drop_first=True)

In [ ]:
finaldata.columns

In [ ]:
features = ['Foot_Traffic_Index', 'Competition_Index', 'Affordability', 'Youth_Percentage', 'Price range']

In [ ]:
features += [col for col in finaldata.columns if 'Cuisine_' in col]

In [ ]:
X = finaldata[features]
y = finaldata['Success_Score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


In [ ]:
# User inputs
user_location = 'Lucknow'
user_cuisine = 'Indian'

# Extract location info
loc_info = location_data[location_data['City'] == user_location].iloc[0]
demo_info = graphics_data[graphics_data['City'] == user_location].iloc[0]
comp_info = competitor_data[competitor_data['City'] == user_location].iloc[0]

# Feature calculations
foot_traffic = loc_info['Foot_Traffic_Index']
avg_rent = loc_info['Avg_Rent_per_sqft']
population = demo_info['Population(2011)']
avg_income = demo_info['Avg_Income']
youth_percent = demo_info['Youth_Percentage']
total_restaurants = comp_info['Total_Restaurants']

competition_index = total_restaurants / population
affordability = avg_income / avg_rent

# Default assumptions for a new restaurant
rating = 4.0
review_count = 50
price_level = '₹₹'
price_mapping = {'₹': 1, '₹₹': 2, '₹₹₹': 3}
price_level_num = price_mapping[price_level]

# Create feature vector
features_dict = {
    'Foot_Traffic_Index': foot_traffic,
    'Competition_Index': competition_index,
    'Affordability': affordability,
    'Youth_Percentage': youth_percent,
    'Price_Level_Num': price_level_num
}

# One-hot encoding for cuisine
for col in X.columns:
    if 'Cuisine_' in col:
        features_dict[col] = 1 if col == f'Cuisine_{user_cuisine}' else 0

# Convert to DataFrame
X_new = pd.DataFrame([features_dict])

